In [3]:
# Googleドライブマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 事前準備

In [4]:
# 提出ファイル名
sub_name = 'pm_lightgbm'

# 削除するカラム
del_columns = ['pm25_mid',
               'City',
               'id',
               'hemisphere',
               'city_month_mean',
               #'city_year_month_mean',
               'country_month_mean',
               'country_month_median',
               'country_year_month_mean',
               #'population',
               #'area',
               #'density'
]

# データの読み込みと確認

In [5]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

# ランダムシードの設定
import random
np.random.seed(1234)
random.seed(1234)

# データの読み込み
df = pd.read_csv('/content/drive/MyDrive/pm/data/processed/processed_df.csv')
submission = pd.read_csv('/content/drive/MyDrive/pm/data/input/submit_sample.csv', header=None)

In [6]:
df

,id,year,month,day,Country,City,lat,lon,co_cnt,co_min,...,population,area,density,hemisphere,season,city_month_mean,city_year_month_mean,country_month_mean,country_year_month_mean,country_month_median
0,1,2019,1,1,Australia,Brisbane,-27.46794,153.02809,3.663562,0.559044,...,2350000,4673.2000,502.867414,1,summer,33.808158,23.133500,44.427941,40.310798,28.690
1,2,2019,1,1,Australia,Darwin,-12.46113,130.84185,3.871201,1.279266,...,140068,3163.8000,44.272078,1,summer,28.573727,22.349633,44.427941,40.310798,28.690
2,3,2019,1,1,Australia,Melbourne,-37.81400,144.96332,2.890372,0.783902,...,4641000,8806.0000,527.027027,1,summer,43.757293,40.569548,44.427941,40.310798,28.690
3,4,2019,1,1,Australia,Newcastle,-32.92953,151.78010,4.158883,1.720263,...,154777,261.8000,591.203209,1,summer,71.098849,55.471935,44.427941,40.310798,28.690
4,5,2019,1,1,Australia,Perth,-31.95224,115.86140,3.871201,1.738534,...,1980000,6418.0000,308.507323,1,summer,45.720391,38.078800,44.427941,40.310798,28.690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249445,249446,2021,12,31,Turkey,Antakya,36.98615,35.32531,4.584967,1.735894,...,1730000,1945.0000,889.460154,0,winter,112.648742,163.016484,84.940667,83.449246,68.519
249446,249447,2021,12,31,Turkey,İzmit,40.19559,29.06013,4.820282,2.924290,...,3101833,10422.0000,297.623585,0,winter,72.303220,60.006387,84.940667,83.449246,68.519
249447,249448,2021,12,31,Turkey,Samsun,41.00500,39.72694,5.017280,0.120446,...,312060,188.8500,1652.422558,0,winter,72.379655,70.846774,84.940667,83.449246,68.519
249448,249449,2021,12,31,United Kingdom,Southend-on-Sea,51.50853,-0.12574,3.091042,0.055435,...,8674000,1572.0000,5517.811705,0,winter,41.818333,46.862000,38.891258,38.630936,30.916


# 特徴量エンジニアリング

In [7]:
# object型の変数の取得
categories = df.columns[df.dtypes == 'object']
print(categories)

# label Encoding
for cat in categories:
    le = LabelEncoder() 
    print(cat)
    
    df[cat].fillna('missing', inplace=True)
    le = le.fit(df[cat])
    df[cat] = le.transform(df[cat])
    # LabelEncoderは数値に変換するだけであるため、最後にastype('category')としておく
    df[cat] = df[cat].astype('category') 

Index(['Country', 'City', 'season'], dtype='object')
Country
City
season


In [8]:
# trainとtestに再分割
train = df[~df['pm25_mid'].isnull()]
test = df[df['pm25_mid'].isnull()]

In [9]:
print(train.columns, len(train.columns))

Index(['id', 'year', 'month', 'day', 'Country', 'City', 'lat', 'lon', 'co_cnt',
       'co_min', 'co_mid', 'co_max', 'co_var', 'o3_cnt', 'o3_min', 'o3_mid',
       'o3_max', 'o3_var', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max',
       'so2_var', 'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'no2_var',
       'temperature_cnt', 'temperature_min', 'temperature_mid',
       'temperature_max', 'temperature_var', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'humidity_var', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'pressure_var',
       'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var', 'dew_cnt', 'dew_min',
       'dew_mid', 'dew_max', 'dew_var', 'pm25_mid', 'population', 'area',
       'density', 'hemisphere', 'season', 'city_month_mean',
       'city_year_month_mean', 'country_month_mean', 'country_year_month_mean',
       'country_month_median'],
      dtype='object') 64


# モデルの構築と評価

In [10]:
# ライブラリのインポート
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from statistics import mean

# K分割する
folds = 20
kf = KFold(n_splits=folds)

# ハイパーパラメータの設定
params = {
    'objective':'regression',
    'random_seed':1234, 
}

# 説明変数と目的変数を指定
X_train = train.drop(del_columns, axis=1)
Y_train = train['pm25_mid']

# 各foldごとに作成したモデルごとの予測値を保存
models = []
rmses = []

for train_index, val_index in kf.split(X_train):
    x_train = X_train.iloc[train_index]
    x_valid = X_train.iloc[val_index]
    y_train = Y_train.iloc[train_index]
    y_valid = Y_train.iloc[val_index]
    
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)    
    
    model = lgb.train(params,
                      lgb_train, 
                      valid_sets=lgb_eval, 
                      num_boost_round=1000, # 学習回数の実行回数
                      early_stopping_rounds=20, # early_stoppingの判定基準
                      verbose_eval=10)
    
    y_pred = model.predict(x_valid, num_iteration=model.best_iteration)
    rmse = np.sqrt(mean_squared_error(y_valid, y_pred))
    print(rmse)
    
    models.append(model)
    rmses.append(rmse)

# 平均AUCを計算する
print(mean(rmses))

# 特徴量重要度の表示
for model in models:
    lgb.plot_importance(model, importance_type='gain',
                        figsize=(10, 15),
                        max_num_features=len(train.columns))

Output hidden; open in https://colab.research.google.com to view.

CV：19.41154144228855

# テストデータの予測

In [11]:
# テストデータの説明変数を指定
X_test = test.drop(del_columns, axis=1)
print(X_test.columns)

# テストデータにおける予測
preds = []

for model in models:
    pred = model.predict(X_test)
    preds.append(pred)

# predsの平均を計算
preds_array = np.array(preds)
preds_mean = np.mean(preds_array, axis=0)

Index(['year', 'month', 'day', 'Country', 'lat', 'lon', 'co_cnt', 'co_min',
       'co_mid', 'co_max', 'co_var', 'o3_cnt', 'o3_min', 'o3_mid', 'o3_max',
       'o3_var', 'so2_cnt', 'so2_min', 'so2_mid', 'so2_max', 'so2_var',
       'no2_cnt', 'no2_min', 'no2_mid', 'no2_max', 'no2_var',
       'temperature_cnt', 'temperature_min', 'temperature_mid',
       'temperature_max', 'temperature_var', 'humidity_cnt', 'humidity_min',
       'humidity_mid', 'humidity_max', 'humidity_var', 'pressure_cnt',
       'pressure_min', 'pressure_mid', 'pressure_max', 'pressure_var',
       'ws_cnt', 'ws_min', 'ws_mid', 'ws_max', 'ws_var', 'dew_cnt', 'dew_min',
       'dew_mid', 'dew_max', 'dew_var', 'population', 'area', 'density',
       'season', 'city_year_month_mean'],
      dtype='object')


# 提出

In [12]:
# 値を置き換え
submission[1] = preds_mean

# CSVファイルの出力
submission.to_csv(f'/content/drive/MyDrive/pm/submit/{sub_name}.csv', header=None, index=False)